# Predicting Rainfall Anomalies in the Amazon
## Model Outline:
- Preprocessing
- Multivariate EOF
- Ridge Regression
- Lasso Regression
- Neural Network Regression

Created by David Gotler, 2021

## Preprocessing
### Define a dataframe class to:
- Load in the datasets:
    - feature dataset (ERA5): variables are CAPE, CIN, geopotential, relative humidity
        - lat: (0,-30), lon: (-40,-80)
    - target dataset (GPCC): variable is precipitation
        - lat: (-5,-25), lon: (-50,-70)
- Detrend the dataset by removing linear trends. We want the dataset to be stationary
- Calculate standardized anomalies by subtracting the monthly mean and dividing by standard deviation. We want centered dataset with unit variance.

In [35]:
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
    
from dataframe import dataframe

# directories
data_dir = 'data_store/'
features_fname = 'ERA5.features.1979-2021.nc'
target_fname = 'precip.mon.total.1x1.v2020.nc'

# Create df object to store our feature and target datasets
df = dataframe(data_dir,features_fname,target_fname)
df.detrend()
df.std_anom()
df.remove_nan()
df.flatten()

/Users/davidgotler/opt/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:166: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/Users/davidgotler/opt/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:166: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/Users/davidgotler/opt/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:166: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/Users/davidgotler/opt/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:166: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/Users/davidgotler/opt/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:166: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/Users/davidgotler/opt/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:166: RankWarnin

In [37]:
from forecast import DirectForecaster,RecursiveForecaster
forecaster = DirectForecaster(LinearRegression(),lags=3,target_months=[9,10,11],steps=3,include_autoreg=True,avg_features=False)
X_train,y_train,X_test,y_test = forecaster.train_test_split([2016,2017,2018,2019],df.target_da,df.features_da)
forecaster.fit(X_train,y_train)
y_pred = forecaster.predict(X_test,y_test)
print('mean absolute error on the test set: {:.3f}%'.format(mean_absolute_error(y_test,y_pred)*100))


<xarray.DataArray 'sample' (sample: 37)>
array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36])
Dimensions without coordinates: sample


ValueError: Could not convert object to NumPy datetime

In [ ]:
model = forecast(df)
model.df.features_ds = model.avg_features()
features_pcs,features_eof,features_var = model.eof_features()
data_splits = model.forecast_split(features_pcs,model.df.target_ds)

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_true,y_pred)

In [ ]:
target_months = np.array([9,10,11])
lags = 3
steps = 3
feature_months = np.arange(target_months.min()-lags,target_months.max()-steps+1)
feature_months

In [ ]:

reg=RegressorWrapper(estimator=LinearRegression(n_jobs=-1))

features_pcs,features_eof,features_var = model.eof_features()
data_splits = model.forecast_split(features_pcs,model.df.target_ds)
# Train on training set,
# (need to add crossvalidation),
# predict and get error on test set
predicted = []

for X_train,y_train,X_test,y_test in data_splits:
    reg.fit(X_train,y=Target()(y_train))
    y_pred = reg.predict(X_test)
    predicted.append(y_pred)

In [ ]:
predicted

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd

# directory 

datadir = 'Data/'

features_fname = datadir+'ERA5.features.1979-2019.JUN-AUG.nc'
target_fname = datadir+'precip.mon.total.1x1.v2020.nc'

# read in features(CAPE,CIN,GEOPOT,RELHUM)
features_ds = xr.open_dataset(features_fname)
features.close()

# select Amazon range for input fields
lat_f_min = -30
lat_f_max = 0
lon_f_min = -80
lon_f_max = -40

features = features.sortby(['lat','lon'])
features = features.sel(lat=slice(lat_f_min,lat_f_max),lon=slice(lon_f_min,lon_f_max))

cape = features.cape
cin = features.cin
geopot = features.z
relhum = features.r
time_jja = pd.DatetimeIndex(features.time.data)

# read in target(PRECIP)
target = xr.open_dataset(target_fname)
target.close()

# Select Amazon range for target field
lat_t_min = -20
lat_t_max = -5
lon_t_min = -65
lon_t_max = -40
year_min = 1979

target_lon_attrs = target.lon.attrs
target['lon'] = xr.where(target['lon']<180,target['lon'],target['lon']-360)
target.lon.attrs = target_lon_attrs
target = target.sortby(['lon','lat'])
target = target.sel(lon=slice(lon_t_min,lon_t_max),lat=slice(lat_t_min,lat_t_max),time=np.logical_and(target['time.year']>=year_min,target['time.season']=='SON'))
precip = target['precip']
time_son = pd.DatetimeIndex(target.time.data)

# features_df = features.to_dataframe(dim_order=['time','lat','lon'])
target

Remove linear trends by fitting linear polynomial

In [ ]:
# Flatten arrays
X = features.to_stacked_array('feature',sample_dims=['time'],name='Features').reset_index('feature')
y = target.to_stacked_array('feature',sample_dims=['time'],name='Target').reset_index('feature')

# Take seasonal means for JJA and SON
# X = X.coarsen(sample=3).mean()
print(X.isnull().sum())
print(y.isnull().sum())
# Remove gridpoints with all nan values
X = X.dropna(dim='feature',how='all')
y = y.dropna(dim='feature',how='all')
print(X.isnull().sum())
print(y.isnull().sum())
# Interpolate the rest of nan and fill w/ 0.0 for any remaining
X = X.fillna()
y = y.interpolate_na(dim='time')
print(X.isnull().sum())
print(y.isnull().sum())
X = X.fillna(0)
y = y.fillna(0)
print(X.isnull().sum())
print(y.isnull().sum())
# Detrend features
X_c = X.polyfit('time',1)
X_trends = xr.polyval(X.sample,X_c)
X = X- X_trends.to_array().squeeze().drop('variable')

# Detrend target
y_c = y.polyfit('time',1)
y_trends = xr.polyval(y.sample,y_c)
y = y- y_trends.to_array().squeeze().drop('variable')
X = (X - X.mean(dim='time'))/X.std(dim='time')
y = (y - y.mean(dim='time'))/y.std(dim='time')

X['time'] = X.time.dt.year
y['time'] = y.time.dt.year

sample = X.time.values

Split into training and test set

In [ ]:
# Define area weights as sqrt(cos(lat)) for X and y grids
X_area_weights = np.sqrt(np.abs(np.cos(np.deg2rad(X.lat.values[np.newaxis,:]))))
y_area_weights = np.sqrt(np.abs(np.cos(np.deg2rad(y.lat.values[np.newaxis,:]))))
# # Choose test set from 2 dry years and 2 wet years
# y_weighted_mean = (y_SON*y_area_weights).mean(dim='feature')
# test_yrs = sample[y_weighted_mean.argsort().values]
# # Must be >1999 to give >=20 years of training data (minimum bound for PCA)
# test_yrs = test_yrs[test_yrs>1989]
# test_yrs = np.block([test_yrs[0:2],test_yrs[-2:]])
# Choose test years as last 4 yrs in dataset
test_yrs = sample[-4:]

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
def time_series_train_test_split(X,y,test_yrs):
    # Split features into training and test set
    X_train = X.drop_sel(sample=test_yrs)
    X_test = X.sel(sample=test_yrs)
    # Split target data into dataset for each month
    y_sep,y_oct,y_nov = y[y.time.dt.month==9],y[y.time.dt.month==10],y[y.time.dt.month==11]
    y_sep_train,y_oct_train,y_nov_train = y_sep.drop_sel(sample=test_yrs),y_oct.drop_sel(sample=test_yrs),y_nov.drop_sel(sample=test_yrs)
    y_sep_test,y_oct_test,y_nov_test = y_sep.sel(sample=test_yrs).rename('y_test_September'),y_oct.sel(sample=test_yrs).rename('y_test_October'),y_nov.sel(sample=test_yrs).rename('y_test_November')
    return X_train,X_test,y_sep_train,y_oct_train,y_nov_train,y_sep_test,y_oct_test,y_nov_test
# Get train,test datasets
y_sep,y_oct,y_nov = y[y.time.dt.month==9],y[y.time.dt.month==10],y[y.time.dt.month==11]
X_train,X_test,y_sep_train,y_oct_train,y_nov_train,y_sep_test,y_oct_test,y_nov_test = time_series_train_test_split(X,y,test_yrs)

# Make cross-validation split for timeseries evaluation
cv = TimeSeriesSplit(12,test_size=1)

In [ ]:
from sklearn.base import BaseEstimator,TransformerMixin

class weight_collumns(BaseEstimator,TransformerMixin):
    def __init__(self,weights):
        self.weights=weights
    def fit(self,X,y=None):
        return self
    def transform(self,X):
        return X*self.weights

In [ ]:
type(test_yrs)==np.ndarray

In [ ]:
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.model_selection import GridSearchCV

def predict_score(model,X_train,y_train,X_test,y_test,label,test_yrs=test_yrs):
    y_pred_list = []
    # Loop through test years'
    if type(test_yrs)==np.ndarray:
        for yr in test_yrs:
            # Fit on leading training data
            model.fit(X_train[X_train.sample<yr],y_train[y_train.sample<yr])
            # Predict test year
            y_pred_res = xr.DataArray(model.predict(X_test.sel(sample=yr).values[np.newaxis,:]).squeeze(),coords=y_test.sel(sample=yr).coords,name=f"y pred {label}")
            y_pred_list.append(y_pred_res)
        # Reformat with xarray
        y_pred = xr.concat(y_pred_list,'sample')
    elif test_yrs=='all':
        model.fit(X_train,y_train)
        y_pred = xr.DataArray(model.predict(X_test).squeeze(),coords=y_test.coords,name=f"y pred {label}")
    mae_test = mean_absolute_error(y_test,y_pred)
    rmse_test = mean_squared_error(y_test,y_pred,squared=False)
    return y_pred,mae_test,rmse_test

def cross_validate(model,X_train,y_train,params,cv):
    # initialize gridsearch cross validation
    reg_cv = GridSearchCV(model,params,scoring='neg_mean_absolute_error',n_jobs=-1,refit=True,cv=cv)
    
    # Train model and get results
    best_estimator = reg_cv.fit(X_train,y_train).best_estimator_
    best_params = reg_cv.best_params_
    best_score = -reg_cv.best_score_
    cv_results = reg_cv.cv_results_
    mae_val = -cv_results['mean_test_score']

    return best_estimator,best_params,best_score,cv_results,mae_val


def validate(model,X,y,params,cv,test_yrs=test_yrs,return_cv_results=False):
    # Get train test split
    X_train,X_test,y_sep_train,y_oct_train,y_nov_train,y_sep_test,y_oct_test,y_nov_test = time_series_train_test_split(X,y,test_yrs)
    print(
        f"Cross-validation using model: {model['regressor']}\n"
    )
    # Cross validate September model
    reg_sep,sep_params,sep_best_score,sep_cv_results,sep_mae_val = cross_validate(model,X_train,y_sep_train,params,cv)
    # Cross validate October model
    reg_oct,oct_params,oct_best_score,oct_cv_results,oct_mae_val = cross_validate(model,X_train,y_oct_train,params,cv)
    # Cross validate November model
    reg_nov,nov_params,nov_best_score,nov_cv_results,nov_mae_val = cross_validate(model,X_train,y_nov_train,params,cv)
    # Choose best model to use for all three months so they have the same parameters
    best_model = [reg_sep,reg_oct,reg_nov][np.argmin([sep_best_score,oct_best_score,nov_best_score])]
    best_params = [sep_params,oct_params,nov_params][np.argmin([sep_best_score,oct_best_score,nov_best_score])]
    # printout
    print(
        f"Minimum Absolute Error of cross-validation:\n"
        f"September: {sep_best_score:.3f}, October: {oct_best_score:.3f}, November: {nov_best_score:.3f}\n"
        f"Best Hyper-parameters:\n"
        f"September: {sep_params}\nOctober: {oct_params}\nNovember: {nov_params}\n"
        f"Mean Absolute Error of cross-validation:\n"
        f"September: {sep_mae_val.mean():.3f} +/- {sep_mae_val.std():.3f}, October: {oct_mae_val.mean():.3f} +/- {oct_mae_val.std():.3f}, November: {nov_mae_val.mean():.3f} +/- {nov_mae_val.std():.3f}\n"
        f"Using parameters: {best_params} for prediction\n"
    )

    # Precict September test set
    y_sep_pred_test,sep_mae_test,sep_rmse_test = predict_score(best_model,X_train,y_sep_train,X_test,y_sep_test,'test September')
    # Precict October test set
    y_oct_pred_test,oct_mae_test,oct_rmse_test = predict_score(best_model,X_train,y_oct_train,X_test,y_oct_test,'test Octtember')
    # Predict November test set
    y_nov_pred_test,nov_mae_test,nov_rmse_test = predict_score(best_model,X_train,y_nov_train,X_test,y_nov_test,'test Novtember')
    print(
        f"Absolute Error on test set:\n"
        f"September: {sep_mae_test:.3f}, October: {oct_mae_test:.3f}, November: {nov_mae_test:.3f}\n"
        f"Root Mean Squared Error on test set:\n"
        f"September: {sep_rmse_test:.3f}, October: {oct_rmse_test:.3f}, November: {nov_rmse_test:.3f}\n"
    )
    # Predict on entire dataset
    y_sep,y_oct,y_nov=y[y.time.dt.month==9],y[y.time.dt.month==10],y[y.time.dt.month==11]
    y_sep_pred,sep_mae,sep_rmse = predict_score(best_model,X,y_sep,X,y_sep,'September',test_yrs='all')
    y_oct_pred,oct_mae,oct_rmse = predict_score(best_model,X,y_oct,X,y_oct,'October',test_yrs='all')
    y_nov_pred,nov_mae,nov_rmse = predict_score(best_model,X,y_nov,X,y_nov,'November',test_yrs='all')
    print(
        f"Absolute Error on entire dataset:\n"
        f"September: {sep_mae:.3f}, October: {oct_mae:.3f}, November: {nov_mae:.3f}\n"
        f"Root Mean Squared Error on entire dataset:\n"
        f"September: {sep_rmse:.3f}, October: {oct_rmse:.3f}, November: {nov_rmse:.3f}\n"
    )
    if return_cv_results==True:
        return y_sep_pred_test,y_oct_pred_test,y_nov_pred_test,y_sep_pred,y_oct_pred,y_nov_pred,sep_cv_results,oct_cv_results,nov_cv_results
    elif return_cv_results==False:
        return y_sep_pred_test,y_oct_pred_test,y_nov_pred_test,y_sep_pred,y_oct_pred,y_nov_pred

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA,FactorAnalysis

In [ ]:
from sklearn.linear_model import Ridge
# Make pipeline to do PCA and train model
pca_ridge = Pipeline([('scalar',StandardScaler()),('weight',weight_collumns(X_area_weights)),('pca',PCA(svd_solver='full')),('varimax',FactorAnalysis(svd_method='lapack',rotation='varimax')),('regressor',Ridge())])
pca_ridge_params = {'pca__n_components':[5,10],'regressor__alpha':[10,1.0,0.1]}
y_sep_pred_test_ridge,y_oct_pred_test_ridge,y_nov_pred_test_ridge,y_sep_pred_ridge,y_oct_pred_ridge,y_nov_pred_ridge = validate(pca_ridge,X,y,pca_ridge_params,cv)

In [ ]:
from sklearn.linear_model import Lasso
pca_lasso = Pipeline([('scalar',StandardScaler()),('weight',weight_collumns(X_area_weights)),('pca',PCA(svd_solver='full')),('varimax',FactorAnalysis(svd_method='lapack',rotation='varimax')),('regressor',Lasso())])
pca_lasso_params = {'pca__n_components':[5,10],'regressor__alpha':[10,1.0,0.1]}
y_sep_pred_test_lasso,y_oct_pred_test_lasso,y_nov_pred_test_lasso,y_sep_pred_lasso,y_oct_pred_lasso,y_nov_pred_lasso = validate(pca_lasso,X,y,pca_lasso_params,cv)

In [ ]:
from sklearn.neural_network import MLPRegressor
pca_MLPR = Pipeline([('scalar',StandardScaler()),('weight',weight_collumns(X_area_weights)),('pca',PCA(svd_solver='full')),('varimax',FactorAnalysis(svd_method='lapack',rotation='varimax')),('regressor',MLPRegressor(shuffle=False,max_iter=1000))])
pca_MLPR_params = {'pca__n_components':[5,10],'regressor__hidden_layer_sizes':[(5,),(10,),(5,5)],'regressor__alpha':[0.001,0.0001]}
y_sep_pred_test_MLPR,y_oct_pred_test_MLPR,y_nov_pred_test_MLPR,y_sep_pred_MLPR,y_oct_pred_MLPR,y_nov_pred_MLPR = validate(pca_MLPR,X,y,pca_MLPR_params,cv)

In [ ]:
from matplotlib import pyplot as plt

def plot_test_pred(y_sep_test,y_oct_test,y_nov_test,y_sep_pred,y_oct_pred,y_nov_pred,test_yrs):
    fig,axs = plt.subplots(4,6)
    for i,yr in zip(np.arange(len(test_yrs)),test_yrs):
        y_sep_test.sel(sample=yr).plot.pcolormesh(x='lon',y='lat',ax=axs[i,0],cmap='rainbow')
    for i,yr in zip(np.arange(len(test_yrs)),test_yrs):
        y_oct_test.sel(sample=yr).plot.pcolormesh(x='lon',y='lat',ax=axs[i,1],cmap='rainbow')
    for i,yr in zip(np.arange(len(test_yrs)),test_yrs):
        y_nov_test.sel(sample=yr).plot.pcolormesh(x='lon',y='lat',ax=axs[i,2],cmap='rainbow')
    for i,yr in zip(np.arange(len(test_yrs)),test_yrs):
        y_sep_pred.sel(sample=yr).plot.pcolormesh(x='lon',y='lat',ax=axs[i,3],cmap='rainbow')
    for i,yr in zip(np.arange(len(test_yrs)),test_yrs):
        y_oct_pred.sel(sample=yr).plot.pcolormesh(x='lon',y='lat',ax=axs[i,4],cmap='rainbow')
    for i,yr in zip(np.arange(len(test_yrs)),test_yrs):
        y_nov_pred.sel(sample=yr).plot.pcolormesh(x='lon',y='lat',ax=axs[i,5],cmap='rainbow')
    return fig,axs
fig,axs = plot_test_pred(y_sep_test,y_oct_test,y_nov_test,y_sep_pred_test_ridge,y_oct_pred_test_ridge,y_nov_pred_test_ridge,test_yrs)

In [ ]:
corr_map_sep_ridge = np.corrcoef(y_sep_pred_ridge,y_sep,rowvar=False)[0:375,375:]
corr_map_oct_ridge = np.corrcoef(y_oct_pred_ridge,y_oct,rowvar=False)[0:375,375:]
corr_map_nov_ridge = np.corrcoef(y_nov_pred_ridge,y_nov,rowvar=False)[0:375,375:]

corr_map_sep_lasso = np.corrcoef(y_sep_pred_lasso,y_sep,rowvar=False)[0:375,375:]
corr_map_oct_lasso = np.corrcoef(y_oct_pred_lasso,y_oct,rowvar=False)[0:375,375:]
corr_map_nov_lasso = np.corrcoef(y_nov_pred_lasso,y_nov,rowvar=False)[0:375,375:]

corr_map_sep_MLPR = np.corrcoef(y_sep_pred_MLPR,y_sep,rowvar=False)[0:375,375:]
corr_map_oct_MLPR = np.corrcoef(y_oct_pred_MLPR,y_oct,rowvar=False)[0:375,375:]
corr_map_nov_MLPR = np.corrcoef(y_nov_pred_MLPR,y_nov,rowvar=False)[0:375,375:]

In [ ]:
plt.pcolormesh(corr_map_sep_lasso)

In [ ]:
import holoviews as hv
from holoviews.util import opts
hv.extension('bokeh')

In [ ]:
y_true_hv = hv.Dataset(y_test,kdims=['time','sample','month','lat','lon'])
y_ridge_hv = hv.Dataset(y_pred_test_ridge,kdims=['time','sample','month','lat','lon'])
y_lasso_hv = hv.Dataset(y_pred_test_lasso,kdims=['time','sample','month','lat','lon'])
y_MLPR_hv = hv.Dataset(y_pred_test_MLPR,kdims=['time','sample','month','lat','lon'])

In [ ]:
y_true_hv.to(hv.Image,kdims=['lat','lon'],groupby=['sample','month'],dynamic=False).layout()

In [ ]:
from matplotlib import pyplot as plt
def plot_error(y_sep_test,y_oct_test,y_nov_test,y_sep_pred,y_oct_pred,y_nov_pred):
    fig,axs = plt.subplots(1,3,figsize=(24,8))
    axs[0].scatter(y_sep_test,y_sep_pred)
    axs[0].plot(np.array([-3,3]),np.array([-3,3]),'k--')
    axs[0].set_xlabel('y true')
    axs[0].set_ylabel('y pred')
    axs[0].set_xlim(-3,3)
    axs[0].set_ylim(-3,3)
    axs[0].set_title('September')

    axs[1].scatter(y_oct_test,y_oct_pred)
    axs[1].plot(np.array([-3,3]),np.array([-3,3]),'k--')
    axs[1].set_xlabel('y true')
    axs[1].set_ylabel('y pred')
    axs[1].set_xlim(-3,3)
    axs[1].set_ylim(-3,3)
    axs[1].set_title('October')

    axs[2].scatter(y_nov_test,y_nov_pred)
    axs[2].plot(np.array([-3,3]),np.array([-3,3]),'k--')
    axs[2].set_xlabel('y true')
    axs[2].set_ylabel('y pred')
    axs[2].set_xlim(-3,3)
    axs[2].set_ylim(-3,3)
    axs[2].set_title('November')
    fig.tight_layout()
    return fig,axs
fig,axs = plot_error(y_sep_test,y_oct_test,y_nov_test,y_sep_pred_test_ridge,y_oct_pred_test_ridge,y_nov_pred_test_ridge)
fig,axs = plot_error(y_sep_test,y_oct_test,y_nov_test,y_sep_pred_test_lasso,y_oct_pred_test_lasso,y_nov_pred_test_lasso)
fig,axs = plot_error(y_sep_test,y_oct_test,y_nov_test,y_sep_pred_test_MLPR,y_oct_pred_test_MLPR,y_nov_pred_test_MLPR)

### Visualize the data

In [ ]:
from matplotlib import pyplot as plt
import matplotlib as mpl

%matplotlib inline

fig,axs = plt.subplots(5,3,figsize=(20,20))
cape.mean(dim=['lat','lon'],keep_attrs=True).plot.line(ax=axs[0,0],x='time',label=cape.name)
cin.mean(dim=['lat','lon'],keep_attrs=True).plot.line(ax=axs[1,0],x='time',label=cin.name)
geopot.mean(dim=['lat','lon'],keep_attrs=True).plot.line(ax=axs[2,0],x='time',label=geopot.name)
relhum.mean(dim=['lat','lon'],keep_attrs=True).plot.line(ax=axs[3,0],x='time',label=relhum.name)
precip.mean(dim=['lat','lon'],keep_attrs=True).plot.line(ax=axs[4,0],x='time',label=precip.name)

cape_trend.mean(dim=['lat','lon'],keep_attrs=True).plot.line('k--',ax=axs[0,0],x='time',label='trend')
cin_trend.mean(dim=['lat','lon'],keep_attrs=True).plot.line('k--',ax=axs[1,0],x='time',label='trend')
geopot_trend.mean(dim=['lat','lon'],keep_attrs=True).plot.line('k--',ax=axs[2,0],x='time',label='trend')
relhum_trend.mean(dim=['lat','lon'],keep_attrs=True).plot.line('k--',ax=axs[3,0],x='time',label='trend')
precip_trend.mean(dim=['lat','lon'],keep_attrs=True).plot.line('k--',ax=axs[4,0],x='time',label='trend')

axs[0,0].set_ylabel(cape.long_name+' ['+cape.units+']')
axs[1,0].set_ylabel(cin.long_name+' ['+cin.units+']')
axs[2,0].set_ylabel(geopot.long_name+' ['+geopot.units+']')
axs[3,0].set_ylabel(relhum.long_name+' ['+relhum.units+']')
axs[4,0].set_ylabel(precip.long_name+' ['+precip.units+']')

cape_anm.mean(dim=['lat','lon'],keep_attrs=True).plot.line(ax=axs[0,1],x='time',label=cape_anm.name)
cin_anm.mean(dim=['lat','lon'],keep_attrs=True).plot.line(ax=axs[1,1],x='time',label=cin_anm.name)
geopot_anm.mean(dim=['lat','lon'],keep_attrs=True).plot.line(ax=axs[2,1],x='time',label=geopot_anm.name)
relhum_anm.mean(dim=['lat','lon'],keep_attrs=True).plot.line(ax=axs[3,1],x='time',label=relhum_anm.name)
precip_anm.mean(dim=['lat','lon'],keep_attrs=True).plot.line(ax=axs[4,1],x='time',label=precip_anm.name)

cape.mean(dim='time',keep_attrs=True).plot(ax=axs[0,2],cmap='Spectral')
cin.mean(dim='time',keep_attrs=True).plot(ax=axs[1,2],cmap='Spectral')
geopot.mean(dim='time',keep_attrs=True).plot(ax=axs[2,2],cmap='Spectral')
relhum.mean(dim='time',keep_attrs=True).plot(ax=axs[3,2],cmap='Spectral')
precip.mean(dim='time',keep_attrs=True).plot(ax=axs[4,2],cmap='Spectral')

# axs[0,2].set_title(cape.long_name+' climatology')
# axs[1,2].set_title(cin.long_name+' climatology')
# axs[2,2].set_title(geopot.long_name+' climatology')
# axs[3,2].set_title(relhum.long_name+' climatology')
# axs[4,2].set_title(precip.long_name+' climatology')

for ax in axs[0:4,0:2].flat:
    ax.grid()
    ax.set_xlim(time_jja.min(),time_jja.max())
    ax.legend()
for ax in axs[4,0:2].flat:
    ax.grid()
    ax.set_xlim(time_son.min(),time_son.max())
    ax.legend()
for ax in axs[:,2].flat:
    ax.grid()

fig.tight_layout()


## Multivariate EOF
- Use eofs library to calculate MEOF on the 4 input fields
- Dimmensionality reduction: retain first 10 PCs to be used as inputs for models

In [ ]:
from eofs.multivariate.standard import MultivariateEof
# Weights arrays
weights_array = np.sqrt(np.abs(np.cos(np.deg2rad(features.lat.data))))[:, np.newaxis]
weights_arrayp = np.sqrt(np.abs(np.cos(np.deg2rad(target.lat.data))))[:, np.newaxis]
ncomp = 10
# MEOF on 4 input variables.
solver = MultivariateEof([cape_anm_JJA.data,cin_anm_JJA.data,geopot_anm_JJA.data,relhum_anm_JJA.data],weights=[weights_array,weights_array,weights_array,weights_array])
meof10_list = solver.eofs(neofs=ncomp,eofscaling=2)
meof10_pcs  = solver.pcs(npcs=ncomp,pcscaling=0)
meof10_pcs = meof10_pcs / meof10_pcs.std()
meof10_vfrc = solver.varianceFraction(neigs=ncomp)

cape_meof10 = meof10_list[0]
cin_meof10 = meof10_list[1]
geopot_meof10 = meof10_list[2]
relhum_meof10 = meof10_list[3]

print("First 10 pcs account for %0.3f %% of the variance" % (np.sum(meof10_vfrc)*100))

In [ ]:
# sm = mpl.cm.ScalarMappable(norm=mpl.colors.Normalize(-1,1),cmap='Spectral')
# fig,axs = ortho_subplots(1,3)
# fig.suptitle('MEOF Structure for CAPE')
# fill = axs[0].contourf(lon_f,lat_f,cape_meof10[0],transform=ccrs.PlateCarree(),levels=100,vmin=-1,vmax=1,cmap='Spectral')
# axs[0].set_title('MEOF 1')
# fill = axs[1].contourf(lon_f,lat_f,cape_meof10[1],transform=ccrs.PlateCarree(),levels=100,vmin=-1,vmax=1,cmap='Spectral')
# axs[1].set_title('MEOF 2')
# fill = axs[2].contourf(lon_f,lat_f,cape_meof10[2],transform=ccrs.PlateCarree(),levels=100,vmin=-1,vmax=1,cmap='Spectral')
# axs[2].set_title('MEOF 3')
# fig.tight_layout()
# fig.colorbar(sm,ax=axs[:],location='bottom',shrink=.6)

# fig,axs = ortho_subplots(1,3)
# fig.suptitle('MEOF Structure for CIN')
# fill = axs[0].contourf(lon_f,lat_f,cin_meof10[0],transform=ccrs.PlateCarree(),levels=100,vmin=-1,vmax=1,cmap='Spectral')
# axs[0].set_title('MEOF 1')
# fill = axs[1].contourf(lon_f,lat_f,cin_meof10[1],transform=ccrs.PlateCarree(),levels=100,vmin=-1,vmax=1,cmap='Spectral')
# axs[1].set_title('MEOF 2')
# fill = axs[2].contourf(lon_f,lat_f,cin_meof10[2],transform=ccrs.PlateCarree(),levels=100,vmin=-1,vmax=1,cmap='Spectral')
# axs[2].set_title('MEOF 3')
# fig.tight_layout()
# fig.colorbar(sm,ax=axs[:],location='bottom',shrink=.6)

# fig,axs = ortho_subplots(1,3)
# fig.suptitle('MEOF Structure for Geopotential')
# fill = axs[0].contourf(lon_f,lat_f,geopot_meof10[0],transform=ccrs.PlateCarree(),levels=100,vmin=-1,vmax=1,cmap='Spectral')
# axs[0].set_title('MEOF 1')
# fill = axs[1].contourf(lon_f,lat_f,geopot_meof10[1],transform=ccrs.PlateCarree(),levels=100,vmin=-1,vmax=1,cmap='Spectral')
# axs[1].set_title('MEOF 2')
# fill = axs[2].contourf(lon_f,lat_f,geopot_meof10[2],transform=ccrs.PlateCarree(),levels=100,vmin=-1,vmax=1,cmap='Spectral')
# axs[2].set_title('MEOF 3')
# fig.tight_layout()
# fig.colorbar(sm,ax=axs[:],location='bottom',shrink=.6)

# fig,axs = ortho_subplots(1,3)
# fig.suptitle('MEOF Structure for Relative Humidity')
# fill = axs[0].contourf(lon_f,lat_f,relhum_meof10[0],transform=ccrs.PlateCarree(),levels=100,vmin=-1,vmax=1,cmap='Spectral')
# axs[0].set_title('MEOF 1')
# fill = axs[1].contourf(lon_f,lat_f,relhum_meof10[1],transform=ccrs.PlateCarree(),levels=100,vmin=-1,vmax=1,cmap='Spectral')
# axs[1].set_title('MEOF 2')
# fill = axs[2].contourf(lon_f,lat_f,relhum_meof10[2],transform=ccrs.PlateCarree(),levels=100,vmin=-1,vmax=1,cmap='Spectral')
# axs[2].set_title('MEOF 3')
# fig.tight_layout()
# fig.colorbar(sm,ax=axs[:],location='bottom',shrink=.6)


fig,ax = plt.subplots(figsize=(16,9),dpi=120)
ax.plot(cape_anm_JJA.time.data,meof10_pcs[:,0],label='PC 1')
ax.plot(cape_anm_JJA.time.data,meof10_pcs[:,1],label='PC 2')
ax.plot(cape_anm_JJA.time.data,meof10_pcs[:,2],label='PC 3')
ax.legend()
ax.set_title('PC Timeseries for MEOF')
ax.set_xlabel('year')
ax.set_xlim(cape_anm_JJA.time.data.min(),cape_anm_JJA.time.data.max())
ax.axhline(c='black',linestyle='--')
fig.tight_layout()

fig,axs = plt.subplots(1,2,figsize=(16,9),dpi=120)
fig.suptitle('Explained Variance of MEOF PC Modes')
axs[0].plot(np.arange(len(meof10_vfrc))+1,meof10_vfrc,'-o')
axs[0].set_title('Fraction of Variance')
axs[0].set_xlim(1,10)
axs[0].set_xlabel('PC Mode')
axs[0].set_ylim(0)
axs[1].plot(np.arange(len(meof10_vfrc))+1,np.cumsum(meof10_vfrc),'-o')
axs[1].set_title('Cumulative Variance')
axs[1].set_xlim(1,10)
axs[1].set_xlabel('PC Mode')
axs[1].set_ylim(0,1);
fig.tight_layout()

## Train Model
- Use TimeSeriesSplit for k-fold cross-validation to
- Train 3 models September, October, November

In [ ]:
from sklearn_xarray import wrap, Target
from sklearn.model_selection import TimeSeriesSplit,GridSearchCV
from sklearn.linear_model import Ridge,Lasso
from sklearn.neural_network import MLPRegressor

In [ ]:
# Split precipitation into sep,oct,nov for 3 models
precip_sep = precip_anm
precip_oct = precip_anm
precip_nov = precip_anm

n = len(years)
n_test = 4
n_train = n-n_test
frac = n_test/n_train
print('(# training samples, # test samples, fraction) = (%i, %i, %0.3f)'%(n_train,n_test,frac))

# test_idx = years == 2010
test_idx = years >= years[n-n_test]
train_idx = np.logical_not(test_idx)

years_train = years[train_idx]
years_test = years[test_idx]

X_train = meof10_pcs[train_idx]
X_test = meof10_pcs[test_idx]

# Save precip shape so we can make it 2D for sklearn
p_shape = precip_sep.shape

y_train_sep = precip_sep[train_idx].reshape(n_train,-1)
y_test_sep = precip_sep[test_idx].reshape(n_test,-1)

y_train_oct = precip_oct[train_idx].reshape(n_train,-1)
y_test_oct = precip_oct[test_idx].reshape(n_test,-1)

y_train_nov = precip_nov[train_idx].reshape(n_train,-1)
y_test_nov = precip_nov[test_idx].reshape(n_test,-1)

y_test_sep_3d = y_test_sep.reshape(n_test,p_shape[1],p_shape[2])
y_test_oct_3d = y_test_oct.reshape(n_test,p_shape[1],p_shape[2])
y_test_nov_3d = y_test_nov.reshape(n_test,p_shape[1],p_shape[2])

print('X_train shape =', X_train.shape)
print('y_train shape =', y_train_nov.shape)
print('X_test shape =', X_test.shape)
print('y_test shape =', y_test_nov.shape)

In [ ]:
# REC curve
tol_max = 2.5
delta = 10*np.finfo(float).eps
tolerance = np.arange(0,tol_max+delta,0.05)

def rec(m,n,tol):
    if type(m)!='numpy.ndarray':
        m=np.array(m)
    if type(n)!='numpy.ndarray':
        n=np.array(n)
    l=m.size
    accuracy = 0
    for i in range(l):
        if np.abs(m[i]-n[i])<=tol:
            accuracy+=1
    return (accuracy/l)

#### Ridge

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge

k = 12
cvsplit = TimeSeriesSplit(n_splits=k,test_size=3)
cvsplit.split(X_train)

parameters = {'alpha':[0.001,0.01,0.1,1,10,20]}

clf = GridSearchCV(Ridge(),parameters,scoring='neg_mean_squared_error',n_jobs=-1,cv=cvsplit)
clf.fit(X_train,y_train_sep)

print("Best parameters set found on September development set:",clf.best_params_)
print("MSE on test set: %0.3f" % -clf.score(X_test,y_test_sep))

clf_sep = clf.best_estimator_

# -------------------

clf.fit(X_train,y_train_oct)

print("\nBest parameters set found on October development set:",clf.best_params_)
print("MSE on test set: %0.3f" % -clf.score(X_test,y_test_oct))

clf_oct = clf.best_estimator_

# ----------------------

clf.fit(X_train,y_train_nov)

print("\nBest parameters set found on November development set:",clf.best_params_)
print("MSE on test set: %0.3f" % -clf.score(X_test,y_test_nov))

clf_nov = clf.best_estimator_


In [ ]:
y_pred_sep_Ridge = clf_sep.predict(meof10_pcs).reshape(p_shape)
y_pred_oct_Ridge = clf_oct.predict(meof10_pcs).reshape(p_shape)
y_pred_nov_Ridge = clf_nov.predict(meof10_pcs).reshape(p_shape)

y_test_pred_sep_Ridge = clf_sep.predict(X_test).reshape(-1,p_shape[1],p_shape[2])
y_test_pred_oct_Ridge = clf_oct.predict(X_test).reshape(-1,p_shape[1],p_shape[2])
y_test_pred_nov_Ridge = clf_nov.predict(X_test).reshape(-1,p_shape[1],p_shape[2])

rec_Ridge_sep = np.empty(len(tolerance))
rec_Ridge_oct = np.empty(len(tolerance))
rec_Ridge_nov = np.empty(len(tolerance))
for t,i in zip(tolerance,range(len(tolerance))):
    rec_Ridge_sep[i] = rec(y_test_pred_sep_Ridge.ravel(),y_test_sep.ravel(),t)
    rec_Ridge_oct[i] = rec(y_test_pred_oct_Ridge.ravel(),y_test_oct.ravel(),t)
    rec_Ridge_nov[i] = rec(y_test_pred_nov_Ridge.ravel(),y_test_nov.ravel(),t)

corr_sep_Ridge = np.empty(p_shape[1:])
for lat in np.arange(len(lat_t)):
    for lon in np.arange(len(lon_t)):
        corr_sep_Ridge[lat,lon] = np.corrcoef(y_pred_sep_Ridge[:,lat,lon],precip_sep[:,lat,lon])[0,1]
corr_oct_Ridge = np.empty(p_shape[1:])
for lat in np.arange(len(lat_t)):
    for lon in np.arange(len(lon_t)):
        corr_oct_Ridge[lat,lon] = np.corrcoef(y_pred_oct_Ridge[:,lat,lon],precip_oct[:,lat,lon])[0,1]
corr_nov_Ridge = np.empty(p_shape[1:])
for lat in np.arange(len(lat_t)):
    for lon in np.arange(len(lon_t)):
        corr_nov_Ridge[lat,lon] = np.corrcoef(y_pred_nov_Ridge[:,lat,lon],precip_nov[:,lat,lon])[0,1]

if n_test == 1:
    fig,axs = plt.subplots(n_test,6,figsize=(16,n_test*3),dpi=120)
    axs[0].contourf(lon_t,lat_t,y_test_sep_3d[0],levels=100,vmin=vmin,vmax=vmax,cmap='Spectral')
    axs[0].set_title('Sep')
    axs[0].set_ylabel(years_test[0])

    axs[1].contourf(lon_t,lat_t,y_test_oct_3d[0],levels=100,vmin=vmin,vmax=vmax,cmap='Spectral')
    axs[1].set_title('Oct')

    axs[2].contourf(lon_t,lat_t,y_test_nov_3d[0],levels=100,vmin=vmin,vmax=vmax,cmap='Spectral')
    axs[2].set_title('Nov')

    axs[3].contourf(lon_t,lat_t,y_test_pred_sep_Ridge[0],levels=100,vmin=vmin,vmax=vmax,cmap='Spectral')
    axs[3].set_title('Sep Prediction')

    axs[4].contourf(lon_t,lat_t,y_test_pred_oct_Ridge[0],levels=100,vmin=vmin,vmax=vmax,cmap='Spectral')
    axs[4].set_title('Oct Prediction')

    axs[5].contourf(lon_t,lat_t,y_test_pred_nov_Ridge[0],levels=100,vmin=vmin,vmax=vmax,cmap='Spectral')
    axs[5].set_title('Nov Prediction')
    plt.colorbar(mpl.cm.ScalarMappable(norm=mpl.colors.Normalize(vmin,vmax),cmap='Spectral'),ax=axs[2:4],location='bottom')
    plt.tight_layout(rect=[0.1,.2,.9,.9])

else:
    fig,axs = plt.subplots(n_test,6,figsize=(16,n_test*3),dpi=120)
    for i,ax in zip(np.arange(n_test),axs.T[0]):
        ax.contourf(lon_t,lat_t,y_test_sep_3d[i],levels=100,vmin=vmin,vmax=vmax,cmap='Spectral')
        ax.set_title('Sep')
        ax.set_ylabel(years_test[i])
    for i,ax in zip(np.arange(n_test),axs[:,1]):
        ax.contourf(lon_t,lat_t,y_test_oct_3d[i],levels=100,vmin=vmin,vmax=vmax,cmap='Spectral')
        ax.set_title('Oct')
    for i,ax in zip(np.arange(n_test),axs[:,2]):
        ax.contourf(lon_t,lat_t,y_test_nov_3d[i],levels=100,vmin=vmin,vmax=vmax,cmap='Spectral')
        ax.set_title('Nov')
    for i,ax in zip(np.arange(n_test),axs[:,3]):
        ax.contourf(lon_t,lat_t,y_test_pred_sep_Ridge[i],levels=100,vmin=vmin,vmax=vmax,cmap='Spectral')
        ax.set_title('Sep Prediction')
    for i,ax in zip(np.arange(n_test),axs[:,4]):
        ax.contourf(lon_t,lat_t,y_test_pred_oct_Ridge[i],levels=100,vmin=vmin,vmax=vmax,cmap='Spectral')
        ax.set_title('Oct Prediction')
    for i,ax in zip(np.arange(n_test),axs[:,5]):
        ax.contourf(lon_t,lat_t,y_test_pred_nov_Ridge[i],levels=100,vmin=vmin,vmax=vmax,cmap='Spectral')
        ax.set_title('Nov Prediction')
    plt.colorbar(mpl.cm.ScalarMappable(norm=mpl.colors.Normalize(vmin,vmax),cmap='Spectral'),ax=axs[-1,2:4],location='bottom')
    plt.tight_layout(rect=[0.1,.2,.9,.9])



## Lasso

In [ ]:
from sklearn.linear_model import Lasso
parameters = {'alpha':[0.0001,0.001,0.01,0.05],}
clf = GridSearchCV(Lasso(),parameters,scoring='neg_mean_squared_error',n_jobs=-1,cv=cvsplit)
clf.fit(X_train,y_train_sep)

print("Best parameters set found on September development set:",clf.best_params_)
print("MSE on test set: %0.3f" % -clf.score(X_test,y_test_sep))

clf_sep = clf.best_estimator_

# --------------------------

clf.fit(X_train,y_train_oct)

print("\nBest parameters set found on October development set:",clf.best_params_)
print("MSE on test set: %0.3f" % -clf.score(X_test,y_test_oct))

clf_oct = clf.best_estimator_

# ---------------------------

clf.fit(X_train,y_train_nov)

print("\nBest parameters set found on November development set:",clf.best_params_)
print("MSE on test set: %0.3f" % -clf.score(X_test,y_test_nov))

clf_nov = clf.best_estimator_

In [ ]:
y_pred_sep_Lasso = clf_sep.predict(meof10_pcs).reshape(p_shape)
y_pred_oct_Lasso = clf_oct.predict(meof10_pcs).reshape(p_shape)
y_pred_nov_Lasso = clf_nov.predict(meof10_pcs).reshape(p_shape)

y_test_pred_sep_Lasso = clf_sep.predict(X_test).reshape(-1,p_shape[1],p_shape[2])
y_test_pred_oct_Lasso = clf_oct.predict(X_test).reshape(-1,p_shape[1],p_shape[2])
y_test_pred_nov_Lasso = clf_nov.predict(X_test).reshape(-1,p_shape[1],p_shape[2])

rec_Lasso_sep = np.empty(len(tolerance))
rec_Lasso_oct = np.empty(len(tolerance))
rec_Lasso_nov = np.empty(len(tolerance))
for t,i in zip(tolerance,range(len(tolerance))):
    rec_Lasso_sep[i] = rec(y_test_pred_sep_Lasso.ravel(),y_test_sep.ravel(),t)
    rec_Lasso_oct[i] = rec(y_test_pred_oct_Lasso.ravel(),y_test_oct.ravel(),t)
    rec_Lasso_nov[i] = rec(y_test_pred_nov_Lasso.ravel(),y_test_nov.ravel(),t)

corr_sep_Lasso = np.empty(p_shape[1:])
for lat in np.arange(len(lat_t)):
    for lon in np.arange(len(lon_t)):
        corr_sep_Lasso[lat,lon] = np.corrcoef(y_pred_sep_Lasso[:,lat,lon],precip_sep[:,lat,lon])[0,1]
corr_oct_Lasso = np.empty(p_shape[1:])
for lat in np.arange(len(lat_t)):
    for lon in np.arange(len(lon_t)):
        corr_oct_Lasso[lat,lon] = np.corrcoef(y_pred_oct_Lasso[:,lat,lon],precip_oct[:,lat,lon])[0,1]
corr_nov_Lasso = np.empty(p_shape[1:])
for lat in np.arange(len(lat_t)):
    for lon in np.arange(len(lon_t)):
        corr_nov_Lasso[lat,lon] = np.corrcoef(y_pred_nov_Lasso[:,lat,lon],precip_nov[:,lat,lon])[0,1]

if n_test == 1:
    fig,axs = plt.subplots(n_test,6,figsize=(16,n_test*3),dpi=120)
    axs[0].contourf(lon_t,lat_t,y_test_sep_3d[0],levels=100,vmin=vmin,vmax=vmax,cmap='Spectral')
    axs[0].set_title('Sep')
    axs[0].set_ylabel(years_test[0])

    axs[1].contourf(lon_t,lat_t,y_test_oct_3d[0],levels=100,vmin=vmin,vmax=vmax,cmap='Spectral')
    axs[1].set_title('Oct')

    axs[2].contourf(lon_t,lat_t,y_test_nov_3d[0],levels=100,vmin=vmin,vmax=vmax,cmap='Spectral')
    axs[2].set_title('Nov')

    axs[3].contourf(lon_t,lat_t,y_test_pred_sep_Lasso[0],levels=100,vmin=vmin,vmax=vmax,cmap='Spectral')
    axs[3].set_title('Sep Prediction')

    axs[4].contourf(lon_t,lat_t,y_test_pred_oct_Lasso[0],levels=100,vmin=vmin,vmax=vmax,cmap='Spectral')
    axs[4].set_title('Oct Prediction')

    axs[5].contourf(lon_t,lat_t,y_test_pred_nov_Lasso[0],levels=100,vmin=vmin,vmax=vmax,cmap='Spectral')
    axs[5].set_title('Nov Prediction')
    plt.colorbar(mpl.cm.ScalarMappable(norm=mpl.colors.Normalize(vmin,vmax),cmap='Spectral'),ax=axs[2:4],location='bottom')
    plt.tight_layout(rect=[0.1,.2,.9,.9])

else:
    fig,axs = plt.subplots(n_test,6,figsize=(16,n_test*3),dpi=120)
    for i,ax in zip(np.arange(n_test),axs.T[0]):
        ax.contourf(lon_t,lat_t,y_test_sep_3d[i],levels=100,vmin=vmin,vmax=vmax,cmap='Spectral')
        ax.set_title('Sep')
        ax.set_ylabel(years_test[i])
    for i,ax in zip(np.arange(n_test),axs[:,1]):
        ax.contourf(lon_t,lat_t,y_test_oct_3d[i],levels=100,vmin=vmin,vmax=vmax,cmap='Spectral')
        ax.set_title('Oct')
    for i,ax in zip(np.arange(n_test),axs[:,2]):
        ax.contourf(lon_t,lat_t,y_test_nov_3d[i],levels=100,vmin=vmin,vmax=vmax,cmap='Spectral')
        ax.set_title('Nov')
    for i,ax in zip(np.arange(n_test),axs[:,3]):
        ax.contourf(lon_t,lat_t,y_test_pred_sep_Lasso[i],levels=100,vmin=vmin,vmax=vmax,cmap='Spectral')
        ax.set_title('Sep Prediction')
    for i,ax in zip(np.arange(n_test),axs[:,4]):
        ax.contourf(lon_t,lat_t,y_test_pred_oct_Lasso[i],levels=100,vmin=vmin,vmax=vmax,cmap='Spectral')
        ax.set_title('Oct Prediction')
    for i,ax in zip(np.arange(n_test),axs[:,5]):
        ax.contourf(lon_t,lat_t,y_test_pred_nov_Lasso[i],levels=100,vmin=vmin,vmax=vmax,cmap='Spectral')
        ax.set_title('Nov Prediction')
    plt.colorbar(mpl.cm.ScalarMappable(norm=mpl.colors.Normalize(vmin,vmax),cmap='Spectral'),ax=axs[-1,2:4],location='bottom')
    plt.tight_layout(rect=[0.1,.2,.9,.9])

## Neural Network

In [ ]:
from sklearn.neural_network import MLPRegressor
parameters = {'hidden_layer_sizes':[5],'alpha':[0.0001,0.001],'activation':['relu']}

clf = GridSearchCV(MLPRegressor(shuffle=False,max_iter=1000),parameters,scoring='neg_mean_squared_error',n_jobs=-1,cv=cvsplit)
clf.fit(X_train,y_train_sep)

print("Best parameters set found on September development set:",clf.best_params_)
print("MSE on test set: %0.3f" % -clf.score(X_test,y_test_sep))

clf_sep = clf.best_estimator_

# --------------------------

clf.fit(X_train,y_train_oct)

print("\nBest parameters set found on October development set:",clf.best_params_)
print("MSE on test set: %0.3f" % -clf.score(X_test,y_test_oct))

clf_oct = clf.best_estimator_

# ---------------------------

clf.fit(X_train,y_train_nov)

print("\nBest parameters set found on November development set:",clf.best_params_)
print("MSE on test set: %0.3f" % -clf.score(X_test,y_test_nov))

clf_nov = clf.best_estimator_

In [ ]:
y_pred_sep_MLPR = clf_sep.predict(meof10_pcs).reshape(p_shape)
y_pred_oct_MLPR = clf_oct.predict(meof10_pcs).reshape(p_shape)
y_pred_nov_MLPR = clf_nov.predict(meof10_pcs).reshape(p_shape)

y_test_pred_sep_MLPR = clf_sep.predict(X_test).reshape(-1,p_shape[1],p_shape[2])
y_test_pred_oct_MLPR = clf_oct.predict(X_test).reshape(-1,p_shape[1],p_shape[2])
y_test_pred_nov_MLPR = clf_nov.predict(X_test).reshape(-1,p_shape[1],p_shape[2])

rec_MLPR_sep = np.empty(len(tolerance))
rec_MLPR_oct = np.empty(len(tolerance))
rec_MLPR_nov = np.empty(len(tolerance))
for t,i in zip(tolerance,range(len(tolerance))):
    rec_MLPR_sep[i] = rec(y_test_pred_sep_MLPR.ravel(),y_test_sep.ravel(),t)
    rec_MLPR_oct[i] = rec(y_test_pred_oct_MLPR.ravel(),y_test_oct.ravel(),t)
    rec_MLPR_nov[i] = rec(y_test_pred_nov_MLPR.ravel(),y_test_nov.ravel(),t)

corr_sep_MLPR = np.empty(p_shape[1:])
for lat in np.arange(len(lat_t)):
    for lon in np.arange(len(lon_t)):
        corr_sep_MLPR[lat,lon] = np.corrcoef(y_pred_sep_MLPR[:,lat,lon],precip_sep[:,lat,lon])[0,1]
corr_oct_MLPR = np.empty(p_shape[1:])
for lat in np.arange(len(lat_t)):
    for lon in np.arange(len(lon_t)):
        corr_oct_MLPR[lat,lon] = np.corrcoef(y_pred_oct_MLPR[:,lat,lon],precip_oct[:,lat,lon])[0,1]
corr_nov_MLPR = np.empty(p_shape[1:])
for lat in np.arange(len(lat_t)):
    for lon in np.arange(len(lon_t)):
        corr_nov_MLPR[lat,lon] = np.corrcoef(y_pred_nov_MLPR[:,lat,lon],precip_nov[:,lat,lon])[0,1]

if n_test == 1:
    fig,axs = plt.subplots(n_test,6,figsize=(16,n_test*3),dpi=120)
    axs[0].contourf(lon_t,lat_t,y_test_sep_3d[0],levels=100,vmin=vmin,vmax=vmax,cmap='Spectral')
    axs[0].set_title('Sep')
    axs[0].set_ylabel(years_test[0])

    axs[1].contourf(lon_t,lat_t,y_test_oct_3d[0],levels=100,vmin=vmin,vmax=vmax,cmap='Spectral')
    axs[1].set_title('Oct')

    axs[2].contourf(lon_t,lat_t,y_test_nov_3d[0],levels=100,vmin=vmin,vmax=vmax,cmap='Spectral')
    axs[2].set_title('Nov')

    axs[3].contourf(lon_t,lat_t,y_test_pred_sep_MLPR[0],levels=100,vmin=vmin,vmax=vmax,cmap='Spectral')
    axs[3].set_title('Sep Prediction')

    axs[4].contourf(lon_t,lat_t,y_test_pred_oct_MLPR[0],levels=100,vmin=vmin,vmax=vmax,cmap='Spectral')
    axs[4].set_title('Oct Prediction')

    axs[5].contourf(lon_t,lat_t,y_test_pred_nov_MLPR[0],levels=100,vmin=vmin,vmax=vmax,cmap='Spectral')
    axs[5].set_title('Nov Prediction')
    plt.colorbar(mpl.cm.ScalarMappable(norm=mpl.colors.Normalize(vmin,vmax),cmap='Spectral'),ax=axs[2:4],location='bottom')
    plt.tight_layout(rect=[0.1,.2,.9,.9])

else:
    fig,axs = plt.subplots(n_test,6,figsize=(16,n_test*3),dpi=120)
    for i,ax in zip(np.arange(n_test),axs.T[0]):
        ax.contourf(lon_t,lat_t,y_test_sep_3d[i],levels=100,vmin=vmin,vmax=vmax,cmap='Spectral')
        ax.set_title('Sep')
        ax.set_ylabel(years_test[i])
    for i,ax in zip(np.arange(n_test),axs[:,1]):
        ax.contourf(lon_t,lat_t,y_test_oct_3d[i],levels=100,vmin=vmin,vmax=vmax,cmap='Spectral')
        ax.set_title('Oct')
    for i,ax in zip(np.arange(n_test),axs[:,2]):
        ax.contourf(lon_t,lat_t,y_test_nov_3d[i],levels=100,vmin=vmin,vmax=vmax,cmap='Spectral')
        ax.set_title('Nov')
    for i,ax in zip(np.arange(n_test),axs[:,3]):
        ax.contourf(lon_t,lat_t,y_test_pred_sep_MLPR[i],levels=100,vmin=vmin,vmax=vmax,cmap='Spectral')
        ax.set_title('Sep Prediction')
    for i,ax in zip(np.arange(n_test),axs[:,4]):
        ax.contourf(lon_t,lat_t,y_test_pred_oct_MLPR[i],levels=100,vmin=vmin,vmax=vmax,cmap='Spectral')
        ax.set_title('Oct Prediction')
    for i,ax in zip(np.arange(n_test),axs[:,5]):
        ax.contourf(lon_t,lat_t,y_test_pred_nov_MLPR[i],levels=100,vmin=vmin,vmax=vmax,cmap='Spectral')
        ax.set_title('Nov Prediction')
    plt.colorbar(mpl.cm.ScalarMappable(norm=mpl.colors.Normalize(vmin,vmax),cmap='Spectral'),ax=axs[-1,2:4],location='bottom')
    plt.tight_layout(rect=[0.1,.2,.9,.9])

In [ ]:
fig,axs = plt.subplots(1,3,figsize=(12,4),dpi=120)
axs[0].set_title("Sep\n")
axs[1].set_title("Oct\n")
axs[2].set_title("Nov\n")
axs[0].plot(tolerance,rec_Ridge_sep,label="Ridge")
axs[0].plot(tolerance,rec_Lasso_sep,label="Lasso")
axs[0].plot(tolerance,rec_MLPR_sep,label="NN")
axs[1].plot(tolerance,rec_Ridge_oct,label="Ridge")
axs[1].plot(tolerance,rec_Lasso_oct,label="Lasso")
axs[1].plot(tolerance,rec_MLPR_oct,label="NN")
axs[2].plot(tolerance,rec_Ridge_nov,label="Ridge")
axs[2].plot(tolerance,rec_Lasso_nov,label="Lasso")
axs[2].plot(tolerance,rec_MLPR_nov,label="NN")
axs[0].set_xlabel("Absolute deviation")
axs[0].set_ylabel("Accuracy")
axs[1].set_xlabel("Absolute deviation")
axs[1].set_ylabel("Accuracy")
axs[2].set_xlabel("Absolute deviation")
axs[2].set_ylabel("Accuracy")
axs[0].grid()
axs[1].grid()
axs[2].grid()
axs[0].legend()
axs[1].legend()
axs[2].legend()
fig.tight_layout()

In [ ]:
fig,axs = plt.subplots(3,3,figsize=(12,6),dpi=120)
axs[0,0].contourf(lon_t,lat_t,corr_sep_Ridge,levels=100,cmap='seismic',vmin=-1,vmax=1)
axs[1,0].contourf(lon_t,lat_t,corr_oct_Ridge,levels=100,cmap='seismic',vmin=-1,vmax=1)
axs[2,0].contourf(lon_t,lat_t,corr_nov_Ridge,levels=100,cmap='seismic',vmin=-1,vmax=1)
axs[0,0].set_title('Ridge')
axs[0,0].set_ylabel('Sep')
axs[1,0].set_ylabel('Oct')
axs[2,0].set_ylabel('Nov')

axs[0,1].contourf(lon_t,lat_t,corr_sep_Lasso,levels=100,cmap='seismic',vmin=-1,vmax=1)
axs[1,1].contourf(lon_t,lat_t,corr_oct_Lasso,levels=100,cmap='seismic',vmin=-1,vmax=1)
axs[2,1].contourf(lon_t,lat_t,corr_nov_Lasso,levels=100,cmap='seismic',vmin=-1,vmax=1)
axs[0,1].set_title('Lasso')

axs[0,2].contourf(lon_t,lat_t,corr_sep_MLPR,levels=100,cmap='seismic',vmin=-1,vmax=1)
axs[1,2].contourf(lon_t,lat_t,corr_oct_MLPR,levels=100,cmap='seismic',vmin=-1,vmax=1)
axs[2,2].contourf(lon_t,lat_t,corr_nov_MLPR,levels=100,cmap='seismic',vmin=-1,vmax=1)
axs[0,2].set_title('NN Regressor')

plt.colorbar(mpl.cm.ScalarMappable(norm=mpl.colors.Normalize(-1,1),cmap='seismic'),ax=axs[:],shrink=0.6)
fig.tight_layout(rect=[0.1,.1,.75,.9])